In [69]:
import json
import shutil
import sys
from pathlib import Path

import numpy as np

In [70]:
quality_check_metrics_file = Path("/home/pauldelseith/HyperDiffusion_semantic_decomposition/siren/experiment_scripts/logs/chair_complete_split_merged_ply/metrics_new.json")
with open(quality_check_metrics_file, "r") as f:
    quality_check_metrics = json.load(f)

In [71]:
filled_difference_fractions = []
filled_difference_fractions_full = []
for value in quality_check_metrics.values():
    # filled_difference_fractions.append(value["split"]["filled_difference_fraction"])
    filled_difference_fractions_full.append(value["full"]["filled_difference_fraction"])
# filled_difference_fractions = np.array(filled_difference_fractions)
filled_difference_fractions_full = np.array(filled_difference_fractions_full)

In [ ]:
filled_difference_fractions.mean()
np.percentile(filled_difference_fractions, [30, 50, 70])

In [72]:
filled_difference_fractions_full.mean()
np.percentile(filled_difference_fractions_full, [30, 50, 80])

array([0.04209126, 0.09009754, 0.20506246])

In [73]:
# filter mlps according to filled_difference_fraction
filled_fraction_threshold = 0.1
mlps_to_keep = []
for key, value in quality_check_metrics.items():
    if value["full"]["filled_difference_fraction"] < filled_fraction_threshold:
        mlps_to_keep.append(key)
print(f"Keeping {len(mlps_to_keep)} out of {len(quality_check_metrics)} MLPs")
good_mlps_list_path = Path("/home/pauldelseith/HyperDiffusion_semantic_decomposition/siren/experiment_scripts/logs/chair_base_seat_sorted_classes/good_shapes.lst")

Keeping 767 out of 1454 MLPs


In [ ]:
with open(good_mlps_list_path, "w") as f:
    f.write("\n".join(mlps_to_keep))

In [ ]:
mlp_weights_path = Path("/home/pauldelseith/HyperDiffusion_semantic_decomposition/mlp_weights/chair_base_seat_sorted_classes")
train_logs_path = Path("/home/pauldelseith/HyperDiffusion_semantic_decomposition/siren/experiment_scripts/logs/chair_base_seat_sorted_classes")
for mlp_weight in train_logs_path.glob("*.pth"):
    shape_id = mlp_weight.stem.split("_")[1]
    if shape_id in mlps_to_keep:
        shutil.copy(mlp_weight, mlp_weights_path / mlp_weight.name)

In [74]:
mesh_dir = Path("/home/pauldelseith/HyperDiffusion_semantic_decomposition/data/partnet/sem_seg_meshes/Chair")
reduced_meshes_dir = Path("/home/pauldelseith/HyperDiffusion_semantic_decomposition/data/partnet/sem_seg_meshes/Chair_reduced_full")
reduced_meshes_dir.mkdir(exist_ok=True, parents=True)
for mesh_file in mesh_dir.glob("*.obj"):
    if len(mesh_file.stem.split("_")) < 3:
        continue
    shape_id = mesh_file.stem.split("_")[0]
    part_name = mesh_file.stem.split("_")[2]
    if shape_id in mlps_to_keep:
        shutil.copy(mesh_file, reduced_meshes_dir / f"{shape_id}_{part_name}.obj")

In [ ]:
meshes_path = Path("/home/pauldelseith/HyperDiffusion_semantic_decomposition/data/partnet/sem_seg_meshes/Chair")
meshes = []
for mesh in meshes_path.glob("*.obj"):
    shape_id = mesh.stem.split("_")[0]
    if shape_id in mlps_to_keep:
        meshes.append(mesh)
with open(meshes_path / "hyper_diff_meshes.lst", "w") as f:
    f.write("\n".join([mesh.name for mesh in meshes]))

In [ ]:
mlps_trained_list_path = Path("/home/pauldelseith/HyperDiffusion_semantic_decomposition/siren/experiment_scripts/logs/chair_base_seat_sorted_classes/mlp_list.lst")
with open(mlps_trained_list_path, "r") as f:
    mlps_trained_list = f.readlines()
shapes_trained_list = []
for line in mlps_trained_list:
    shapes_trained_list.append(line.split("_")[1]) 
with open(mlps_trained_list_path.parent / "shapes_trained.lst", "w") as f:
    f.write("\n".join(shapes_trained_list))